In [14]:
from pathlib import Path
from typing import List

import pandas as pd
import numpy as np

YEAR = 2022

In [5]:
!pip install -q advent-of-code-data

!mkdir inputs
!aocd 1 $YEAR > inputs/01_calories.txt

You should consider upgrading via the 'C:\AnsysDev\Python39\python.exe -m pip install --upgrade pip' command.


# 1st December

In [15]:
def get_calories(path: Path) -> List[int]:
    """Retrieve calories carried by each elf"""
    content = Path(path).read_text(encoding="utf-8")
    return [
        sum(int(line) for line in block.strip().split("\n"))
        for block in content.split("\n\n")
    ]


calories = pd.Series(get_calories("inputs/01_calories.txt"), name="calories")
calories


0      55684
1      43399
2      48846
3      48058
4      52061
       ...  
245    22318
246    48674
247    43489
248    47659
249    50009
Name: calories, Length: 250, dtype: int64

In [11]:
# Get the top calories
calories.max()

69693

In [12]:
# Get the sum of calories from top 3
calories.sort_values(ascending=False).head(3).sum()

200945

## Retrieving Data automatically

In [17]:
from aocd import get_data
YEAR = 2022

data = get_data(day=1, year=YEAR)
data[:100]

'10130\n9819\n4257\n8400\n10994\n3758\n8326\n\n9002\n15602\n1193\n6805\n10797\n\n9435\n6559\n11512\n12051\n2687\n2850\n37'

In [18]:
def get_calories_auto() -> List[int]:
    """Retrieve calories carried by each elf"""
    data = get_data(day=1, year=YEAR)
    return [
        sum(int(line) for line in block.strip().split("\n"))
        for block in data.split("\n\n")
    ]


calories = pd.Series(get_calories_auto(), name="calories")
calories


0      55684
1      43399
2      48846
3      48058
4      52061
       ...  
245    22318
246    48674
247    43489
248    47659
249    50009
Name: calories, Length: 250, dtype: int64